In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("2017_2019.csv")
df.drop(['Unnamed: 18'],inplace=True,axis=1)

In [3]:
dates = set()
for i,data in df.iterrows():
    dates.add(pd.to_datetime( 
                             str(data['Year'])[:-2]+'-'+str(data['Month'])[:-2] + '-' + str(data['Day'])[:-2],
                             format="%Y-%m-%d"
                             ))

In [4]:
def get_day(df:pd.DataFrame,day:pd.Timestamp) -> pd.DataFrame:
    
    return df.loc[ (df ['Year'] == day.year) & 
                  (df['Day'] == day.day) &
                  (df['Month'] == day.month)]
    
def get_records_night(df:pd.DataFrame,day:pd.Timestamp):
    records_beafore_df = get_n_days_beafore(df,day,1)
    records_beafore_df = records_beafore_df.loc[(records_beafore_df.Hour > 12)  & (records_beafore_df["Clearsky DNI"] != 0)]
    
    records_df = get_day(df,day)
    records_df = records_df.loc[(records_df.Hour < 12) &( records_df.DNI == 0) &(records_df["Clearsky DNI"] != 0)]
    
    return(pd.concat([records_beafore_df,records_df]))
    
    
  
def get_n_days_beafore(df:pd.DataFrame,day:pd.Timestamp,n:int):
    day_beafore = day - pd.offsets.Day(n)
    return get_day(df,day_beafore)

def get_working_day(df:pd.DataFrame,day:pd.Timestamp):
    records = get_day(df,day)
    return records.loc[(records.DNI != 0) | (records["Clearsky DNI"] != 0)]

    
    

In [5]:
_ = pd.Series( list(dates)).sort_values()
get_records_night(df,_.iloc[2])

,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,DHI,DNI,GHI,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Wind Speed
148,2017,1,2,13,0,7.5,69,738,296,-1.7,69,738,296,52.30,72.09,0.13,1000,0.9
149,2017,1,2,13,15,7.2,65,702,258,-1.7,65,702,258,53.38,74.03,0.13,1000,0.8
150,2017,1,2,13,30,6.8,60,658,219,-1.7,60,658,219,54.86,76.07,0.13,1000,0.8
151,2017,1,2,13,45,6.5,54,610,179,-2.4,54,610,179,52.89,78.21,0.13,1000,0.8
152,2017,1,2,14,0,6.1,48,542,138,-2.4,48,542,138,54.36,80.44,0.13,1000,0.7
153,2017,1,2,14,15,6.0,40,455,98,-2.4,40,455,98,54.74,82.74,0.13,1000,0.7
154,2017,1,2,14,30,5.9,31,342,60,-2.4,31,342,60,55.12,85.11,0.13,1000,0.6
155,2017,1,2,14,45,5.7,19,199,28,-3.1,19,199,28,52.99,87.49,0.13,1000,0.6
215,2017,1,3,5,45,3.8,19,155,25,0.7,9,0,9,80.07,87.59,0.13,1000,3.0
216,2017,1,3,6,0,4.3,31,285,55,0.7,20,0,20,77.31,85.21,0.13,1000,3.1


In [6]:
df[df.isna().any(axis=1)]

,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,DHI,DNI,GHI,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Wind Speed


In [19]:
records_df =pd.DataFrame()
for idx,date in enumerate( pd.Series( list(dates)).sort_values().values):
    
    date = pd.to_datetime(date)
    day = get_working_day(df,date)
    record = pd.DataFrame([{'date':date}])
    
    record['month'] = date.month
    record['len_day'] = len(day)
    record['temp_mean'] = day.Temperature.mean()
    record['press_mean'] = day.Pressure.mean()
    record['wind_mean'] = day["Wind Speed"].mean()
    
    
    night = get_records_night(df,date)
    
    if idx == 0:
        record['len_night'] = 0
    else:
        record['len_night'] = len(night)
        record['night_temp_mean'] = night.Temperature.mean()
        record['night_press_mean'] = night.Pressure.mean()
        record['night_wind_mean'] = night["Wind Speed"].mean()
    
    record['Y'] = (day["Clearsky DNI"].sum() + day['DNI'].sum()) / record['len_day']
    records_df = pd.concat([records_df,record])
records_df

,date,month,len_day,temp_mean,press_mean,wind_mean,len_night,Y,night_temp_mean,night_press_mean,night_wind_mean
0,2017-01-01,1,37,4.910811,1001.648649,1.878378,0,1397.243243,NaN,NaN,NaN
0,2017-01-02,1,37,6.035135,1000.918919,1.335135,11,1391.621622,3.863636,1000.727273,1.709091
0,2017-01-03,1,37,8.983784,999.756757,2.970270,14,1068.054054,5.828571,1000.285714,1.835714
0,2017-01-04,1,37,10.167568,996.621622,3.432432,15,685.891892,8.826667,998.400000,2.533333
0,2017-01-05,1,37,14.021622,988.594595,6.070270,26,538.972973,12.834615,990.846154,5.188462
...,...,...,...,...,...,...,...,...,...,...,...
0,2019-12-27,12,37,8.959459,996.324324,3.143243,8,1206.540541,9.762500,989.875000,3.962500
0,2019-12-28,12,37,8.727027,991.810811,1.532432,22,595.216216,8.468182,993.545455,1.740909
0,2019-12-29,12,37,7.818919,992.486486,4.994595,19,636.756757,6.684211,992.157895,3.405263
0,2019-12-30,12,37,7.432432,987.621622,6.756757,33,566.675676,7.439394,988.303030,6.748485


In [20]:
records_df[records_df.isna().any(axis=1)]

,date,month,len_day,temp_mean,press_mean,wind_mean,len_night,Y,night_temp_mean,night_press_mean,night_wind_mean
0,2017-01-01,1,37,4.910811,1001.648649,1.878378,0,1397.243243,NaN,NaN,NaN


In [21]:
def get_Data_n_records(df,num):
    X = []
    Y = []
    for i in range(len(df)):
        if i < num+1:
            pass
        else:
            X.append(df.iloc[i-num-1:i-1].values)
            Y.append(df.iloc[i].Y)
    return np.array(X),np.array(Y)

X,Y = get_Data_n_records(records_df,3)

In [22]:
import pandas as pd
import numpy as np 
import keras as kr
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
from sklearn.preprocessing import normalize

In [23]:
records_df.drop(["date"], inplace= True,axis= 1)
records_df.dropna(inplace=True)

In [24]:
records_df[records_df.isna().any(axis=1)]

,month,len_day,temp_mean,press_mean,wind_mean,len_night,Y,night_temp_mean,night_press_mean,night_wind_mean


In [43]:
num = 4
X,Y = get_Data_n_records(records_df,num)

Y = pd.Series(Y)
Y=(Y-Y.min())/(Y.max()-Y.min())
x_train,x_test,y_train,y_test = train_test_split(X,Y)

# regr = RandomForestRegressor(random_state=0)
# regr.fit(x_train, y_train)

mlp_model = kr.Sequential([
    kr.layers.Flatten(input_shape = (num,10)),
    kr.layers.Dense(15, activation = 'relu'),
    kr.layers.Dense(1, activation = 'sigmoid')
])

mlp_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                  loss=tf.keras.losses.MeanAbsoluteError(reduction="auto", name="mean_absolute_error"), 
                  metrics=tf.keras.metrics.Recall()
)
history = mlp_model.fit(x_train, y_train, batch_size=128, epochs=100)

Epoch 1/100
7/7 [==============================] - 0s 1ms/step - loss: 0.4480 - recall_17: 0.5123
Epoch 2/100
7/7 [==============================] - 0s 2ms/step - loss: 0.4374 - recall_17: 0.5588
Epoch 3/100
7/7 [==============================] - 0s 1ms/step - loss: 0.4305 - recall_17: 0.5919
Epoch 4/100
7/7 [==============================] - 0s 1ms/step - loss: 0.4273 - recall_17: 0.6140
Epoch 5/100
7/7 [==============================] - 0s 2ms/step - loss: 0.4272 - recall_17: 0.6176
Epoch 6/100
7/7 [==============================] - 0s 1ms/step - loss: 0.4255 - recall_17: 0.6152
Epoch 7/100
7/7 [==============================] - 0s 1ms/step - loss: 0.4323 - recall_17: 0.5858
Epoch 8/100
7/7 [==============================] - 0s 2ms/step - loss: 0.4335 - recall_17: 0.5723
Epoch 9/100
7/7 [==============================] - 0s 2ms/step - loss: 0.4333 - recall_17: 0.5833
Epoch 10/100
7/7 [==============================] - 0s 2ms/step - loss: 0.4331 - recall_17: 0.5858
Epoch 11/100
7/7 [=

In [45]:
y_test.values,mlp_model.predict(x_test)

9/9 [==============================] - 0s 797us/step


(array([0.78102351, 0.69226237, 0.26664779, 0.76397125, 0.80023921,
        0.31661576, 0.72286267, 0.54213372, 0.22824308, 0.4292905 ,
        0.66471744, 0.72249104, 0.65834026, 0.49379816, 0.58192926,
        0.82384997, 0.30193413, 0.44558502, 0.29666016, 0.86210491,
        0.77732917, 0.84440482, 0.27658784, 0.68318856, 0.67799099,
        0.41785694, 0.22942829, 0.77394675, 0.38085311, 0.8173175 ,
        0.23814315, 0.4604204 , 0.11305763, 0.58653832, 0.27545432,
        0.91276381, 0.66742647, 0.        , 0.13524211, 0.78766393,
        0.18628029, 0.89582981, 0.65505338, 0.60286243, 0.61805788,
        0.52892222, 0.78905921, 0.19531834, 0.29470606, 0.32235889,
        0.55632598, 0.62817577, 0.60776447, 0.92076803, 0.34828885,
        0.52531722, 0.3487377 , 0.21806288, 0.802199  , 0.60945199,
        0.30291054, 0.19049232, 0.52521622, 0.72915236, 0.86095193,
        0.37705744, 0.74581484, 0.96206593, 0.45548072, 0.38076155,
        0.80465435, 0.35954943, 0.54014511, 0.30